In [2]:
import pandas as pd
from typing import Dict, Tuple, List, Optional
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib.figure import Figure

# Consigne

- Fill in the code of this Notebook wherever you see "..." using the comments.

- Don’t hesitate to take ownership of the existing code by running parts of it in separate cells.

In [69]:
df_CAC = pd.read_excel("data_CAC.xlsx",sheet_name="Prices",index_col="Dates").loc["2011-12-30":"2016-12-30",:]

In [70]:
df_CAC

,OR FP Equity,DG FP Equity,STLA FP Equity,URW NA Equity,TTE FP Equity,AI FP Equity,DSY FP Equity,CS FP Equity,ERF FP Equity,BNP FP Equity,...,VIE FP Equity,SAN FP Equity,GLE FP Equity,SU FP Equity,AIR FP Equity,LR FP Equity,ACA FP Equity,RI FP Equity,STM FP Equity,WLN FP Equity
Dates,,,,,,,,,,,,,,,,,,,,,
2011-12-30,80.70,33.760,NaN,138.90,39.500,63.5925,6.193,10.045,5.633,30.350,...,8.1656,56.75,17.205,40.680,24.150,24.850,4.360,71.66,4.591,NaN
2012-01-02,81.80,34.450,NaN,137.80,40.000,64.6170,6.285,10.430,5.750,31.290,...,8.6149,57.42,17.685,41.560,24.500,25.160,4.433,72.47,4.703,NaN
2012-01-03,82.12,34.755,NaN,138.25,40.445,64.8432,6.263,10.580,5.937,31.335,...,8.5194,57.29,17.600,43.260,24.500,25.305,4.433,72.06,4.946,NaN
2012-01-04,81.33,34.550,NaN,136.60,40.135,64.4308,6.182,10.400,5.750,30.650,...,8.1829,56.21,17.000,42.080,24.565,25.160,4.300,71.62,4.791,NaN
2012-01-05,81.42,34.060,NaN,134.45,39.860,63.2998,6.195,9.890,5.661,29.000,...,7.9168,55.50,16.080,42.505,24.890,24.770,4.076,71.53,4.726,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-26,172.55,64.480,NaN,221.55,48.360,86.9421,14.372,24.050,40.090,61.170,...,15.3737,75.86,46.810,65.150,62.900,54.020,11.905,102.05,10.745,26.215
2016-12-27,172.70,64.530,NaN,222.90,48.375,87.1074,14.400,24.165,40.235,60.680,...,15.4990,76.22,46.810,65.420,62.920,54.120,11.880,102.55,10.840,26.505
2016-12-28,172.40,64.190,NaN,220.20,48.720,87.1901,14.486,23.995,40.540,60.650,...,15.4460,76.15,46.740,65.610,62.720,54.000,11.880,102.35,10.795,26.570


In [ ]:
def calculer_rendements(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute daily returns for all stocks
    
    Parameters:
        df (pd.DataFrame): DataFrame with dates as index and tickers as columns
    
    Returns:
        pd.DataFrame: a new DataFrame containing the daily returns
    """
    rendements = ...
    return rendements.dropna()

In [ ]:
def calculer_stats_descriptives(rendements: pd.DataFrame) -> pd.DataFrame:
    """
    Compute descriptive statistics for each stock
    
    Parameters:
        rendements (pd.DataFrame): DataFrame of daily returns
    
    Returns:
        pd.DataFrame: a new DataFrame containing the descriptive statistics
    """
    stats = pd.DataFrame()
    stats['Annualized mean return'] = ...
    stats['Annualized volatility'] = ...
    stats['Skewness'] = ...
    stats['Kurtosis'] = ...
    stats['Minimum return'] = ...
    stats['Maximum return'] = ...
    return stats


In [ ]:
def calculer_matrice_correlation(rendements: pd.DataFrame) -> pd.DataFrame:
    """
    Compute the correlation matrix between stocks
    
    Parameters:
        rendements (pd.DataFrame): DataFrame of daily returns
    
    Returns:
        pd.DataFrame: Correlation matrix
    """
    return ...


In [66]:
def calculer_capm(rendements: pd.DataFrame, rf: float = 0.01) -> pd.DataFrame:
    """
    Compute CAPM for each stock using an equally-weighted market return estimate without rebalancing.
    
    Parameters:
        rendements (pd.DataFrame): DataFrame of daily returns.
        rf (float): Annualized risk-free rate.
    
    Returns:
        pd.DataFrame: DataFrame containing betas and expected returns.
    """
    # number of stocks
    nb_actions = ...
    # create a pd.Series with the same indices as rendements and weight values
    # All weights must be equal (equally-weighted portfolio)
    poids_marche = pd.Series(1 / nb_actions, index=rendements.columns)
    
    rm_quotidien = ...  # create an empty pd.Series with the same indices as rendements
    valeurs_portefeuille = [100]  # initial value of the market portfolio

    for i, jour in enumerate(rendements.index):
        if i == 0:
            # For the first day, initialize the market return with the vector of initial weights
            rm_quotidien[jour] = (rendements.loc[jour] * poids_marche).sum()
        else:
            # Update portfolio values without rebalancing, i.e compute the new value of the portfolio based on the weighted performance of the stocks.
            valeurs_portefeuille.append(...)
            # Compute the market return for the day
            rm_quotidien[jour] = ...
            
        # compute weight drift (weights drift according to asset performance)
        poids_marche = ...
        poids_marche /= ...  # normalize poids_marche so that the sum of weights is 1
    
    # Compute total annualized return (note: geometric returns)
    rm_annuel = ...
    
    # Compute betas
    betas = {}
    for stock in rendements.columns:
        covar = ...
        var_rm = ...
        beta = ...
        betas[colonne] = ...
    
    # Compute risk premium (excess market return over the risk-free rate)
    risk_premium = ...
    
    # Compute expected returns
    expected_returns = {}
    for action, beta in betas.items():
        expected_returns[action] = ...
    
    return pd.DataFrame({
        'Beta': betas,
        'Expected return': expected_returns,
        'Risk premium': risk_premium,
        'Market return': rm_annuel
    })


In [1]:
def optimisation_portefeuille(rendements, rf=0.01, target_return=None):
    """
    Compute the minimum variance portfolio in the Markowitz sense.
    If a target return is provided as input, the function computes an efficient portfolio under the constraint of a target return.
    """
    # number of assets
    n_assets = ...
    # NOTE: assumption that returns are small and iid: approximation of the geometric mean by the arithmetic mean
    mu = ...  # annualized mean (252 * mean of the daily return)
    cov_matrix = ...  # annualized variance-covariance matrix
    
    def portfolio_variance(weights):
        return ...
    
    def portfolio_expected_return(weights):
        return ...
    
    # declaration of optimization constraints (portfolio 100% invested + target return if provided by the user)
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    ]
    
    if target_return is not None:
        constraints.append(
            {'type': 'eq', 'fun': lambda x: portfolio_expected_return(x) - target_return}
        )
    
    # bounds for the asset weights
    bounds = tuple((0, 1) for _ in range(n_assets))
    weights_init = np.array([1/n_assets] * n_assets)
    
    result = minimize(portfolio_variance, weights_init,
                     method='SLSQP',
                     bounds=bounds,
                     constraints=constraints)
    
    weights_opt = result.x
    rendement_portefeuille = ...
    volatilite_portefeuille = ...
    
    return {
        'Weights': pd.Series(weights_opt, index=rendements.columns),
        'Expected return': rendement_portefeuille,
        'Volatility': volatilite_portefeuille
    }


In [3]:
def tracer_frontiere_efficiente(
    rendements: pd.DataFrame, 
    rf: float = 0.01, 
    n_portfolios: int = 50
) -> Figure:
    """
    Plot the efficient frontier + the minimum variance and maximum Sharpe ratio portfolios
    
    Parameters:
        rendements (pd.DataFrame): DataFrame of daily returns
        rf (float): Annualized risk-free rate
        n_portfolios (int): Number of portfolios to compute
    
    Returns:
        Figure: Matplotlib figure of the efficient frontier
    """
    # create a linspace of length n_portfolios between the minimum annualized mean return and the maximum annualized mean return
    returns_range = np.linspace(
        rendements.mean().min() * 252,  # lowest expected return
        rendements.mean().max() * 252,  # highest expected return
        n_portfolios  # number of points (50 by default)
    )
    
    volatilities = []
    returns_list = []
    sharpe_ratios = []
    
    min_volatility = float('inf')
    max_sharpe_ratio = -float('inf')
    min_volatility_portfolio = None
    max_sharpe_portfolio = None
    
    for target_return in returns_range:
        # for each mean return, estimate a min-variance portfolio under the constraint of a target return
        # store the result in result
        result = ...
        # retrieve optimal weights
        weights = ...
        # retrieve the volatility of this optimal portfolio
        portfolio_vol = ...
        # store the return of the optimal portfolio for a given target return
        portfolio_return = target_return
        
        # compute the Sharpe ratio for this portfolio
        sharpe_ratio = ...
        
        # add the values into volatilities, returns_list and sharpe_ratios
        ...
        ...
        ...
        
        # Check if this is the minimum variance portfolio
        # if yes, update min_volatility and min_volatility_portfolio (a tuple with volatility and portfolio return)
        if ... < ... :
            min_volatility = ...
            min_volatility_portfolio = (..., ...)
        
        # Check if this is the portfolio with the maximum Sharpe ratio
        # if yes, update max_sharpe_ratio and max_sharpe_portfolio (a tuple with volatility and portfolio return)
        if ... > ... :
            max_sharpe_ratio = ...
            max_sharpe_portfolio = (..., ...)
    
    # Split the frontier into two segments, below and above the min-variance portfolio
    min_index = volatilities.index(min_volatility)
    volatilities_below = volatilities[:min_index + 1]
    returns_below = returns_list[:min_index + 1]
    volatilities_above = volatilities[min_index:]
    returns_above = returns_list[min_index:]
    
    plt.figure(figsize=(12, 8))
    # Plot the part below min-variance in red and the part above in blue
    plt.plot(volatilities_below, returns_below, 'r-', label='Envelope of feasible portfolios')
    plt.plot(volatilities_above, returns_above, 'b-', label='Efficient frontier')
    
    # Add the minimum variance portfolio (green point)
    if min_volatility_portfolio is not None:
        plt.scatter(min_volatility_portfolio[0], min_volatility_portfolio[1], 
                    c='green', marker='o', s=100, label='Minimum variance portfolio')
    
    # add the maximum Sharpe ratio portfolio (orange point)
    if max_sharpe_portfolio is not None:
        plt.scatter(max_sharpe_portfolio[0], max_sharpe_portfolio[1], 
                    c='orange', marker='o', s=100, label='Tangent portfolio (max Sharpe)')
    
    # add a scatter plot for the stocks in the volatility/annualized return space
    vols_actions = ...
    returns_actions = ...
    plt.scatter(vols_actions, returns_actions, c='red', marker='o', label='Individual stocks')
    
    for i, txt in enumerate(rendements.columns):
        plt.annotate(txt, (vols_actions[i], returns_actions[i]))
    
    plt.xlabel('Annualized volatility')
    plt.ylabel('Expected annualized return')
    plt.title('Efficient Frontier, Minimum Variance Portfolio and Maximum Sharpe Ratio')
    plt.legend()
    plt.grid(True)
    
    return plt.gcf()  # .gcf() means "get current figure"


In [2]:
def graphique_composition_portefeuille(weights):
    """
    Display a bar chart showing the composition of the portfolio,
    sorted from the largest to the smallest weight.
    
    Parameters:
        weights (pd.Series): Vector of asset weights in the portfolio.
    """
    
    # Sort weights from largest to smallest
    sorted_weights = ...
    
    # Plot the chart
    plt.figure(figsize=(10, 6))
    sorted_weights.plot(kind='bar', color='skyblue', edgecolor='black')
    
    # Add titles and labels
    plt.title('Portfolio Composition', fontsize=16)
    plt.xlabel('Assets', fontsize=14)
    plt.ylabel('Weights', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    
    # Show the chart
    plt.tight_layout()
    plt.show()


In [5]:
def main() -> Dict[str, pd.DataFrame]:
    """
    Main function that executes the complete analysis
    
    Returns:
        Dict[str, pd.DataFrame]: Dictionary containing all analysis results
    """
    
    r_f = 0.01
    
    # Create a synthetic DataFrame
    df = df_CAC.copy(deep=True)
    
    print("1. Compute returns")
    print("-" * 50)
    rendements = ...
    
    print("\n2. Descriptive statistics")
    print("-" * 50)
    stats = ...
    print("\nStatistics by stock:")
    print(stats)
    
    print("\n3. Correlation matrix")
    print("-" * 50)
    corr = ...
    
    print("\n4. CAPM analysis")
    print("-" * 50)
    capm_results = ...
    print("\nCAPM results:")
    print(capm_results)
    
    print("\n5. Optimal portfolio")
    print("-" * 50)
    ptf_optimal = ...
    print("\nOptimal weights:")
    print(ptf_optimal)
    # display the chart of optimal portfolio weights
    ...
    
    print("\n6. Efficient frontier visualization")
    print("-" * 50)
    # display the efficient frontier
    fig = ...
    plt.show()

    return {
        'returns': rendements,
        'stats': stats,
        'capm': capm_results,
        'weights_optimal': ptf_optimal["Weights"]
    }


In [7]:
main()